In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [25]:
#Data set (tips data set)
#dependent feature will be total bill
df=sns.load_dataset('tips')

In [26]:
df.isnull().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

In [27]:
# 1. Remove duplicates
df = df.drop_duplicates()

# 2. Handle missing values (if any)
df = df.dropna()  # or fillna before split if it's rule-based

# 3. Encode simple binary columns manually (safe)
df['sex'] = df['sex'].map({'Male': 1, 'Female': 0})
df['smoker'] = df['smoker'].map({'Yes': 1, 'No': 0})
df['time'] = df['time'].map({'Lunch': 0, 'Dinner': 1})


In [28]:
x=df.iloc[: ,1:7]
y=df['total_bill']

In [29]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=42)

In [30]:
#encoing using sklearn function
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
le1=LabelEncoder()
le2=LabelEncoder()

In [31]:
ct = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(drop='first'), ['day'])
    ],
    remainder='passthrough'
)


In [32]:
x_train=ct.fit_transform(x_train)

In [33]:
x_test=ct.transform(x_test)

In [36]:
from sklearn.svm import SVR
svr=SVR()

In [37]:
svr.fit(x_train,y_train)

,kernel,'rbf'
,degree,3
,gamma,'scale'
,coef0,0.0
,tol,0.001
,C,1.0
,epsilon,0.1
,shrinking,True
,cache_size,200
,verbose,False
,max_iter,-1


In [38]:
y_pred=svr.predict(x_test)

In [39]:
from sklearn.metrics import r2_score,mean_absolute_error
print(mean_absolute_error(y_test,y_pred))
print(r2_score(y_test,y_pred))

4.660559025907575
0.39537989683307306


In [42]:
#hyperparameter tuning using grid search cv
from sklearn.model_selection import GridSearchCV

In [50]:
param_grid={'C':[0.1,1,0.01,0.001,0.001],
            'gamma':[1,0.1,0.01,0.001,0.0001],
            'kernel':['rbf',]}

In [51]:
grid=GridSearchCV(SVR(),param_grid,refit=True,verbose=3)
#fitting with grid
grid.fit(x_train,y_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.048 total time=   0.0s
[CV 2/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.112 total time=   0.0s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.008 total time=   0.0s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.006 total time=   0.0s
[CV 5/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.079 total time=   0.0s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.060 total time=   0.0s
[CV 2/5] END .....C=0.1, gamma=0.1, kernel=rbf;, score=-0.006 total time=   0.0s
[CV 3/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.106 total time=   0.0s
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.142 total time=   0.0s
[CV 5/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.017 total time=   0.0s
[CV 1/5] END ....C=0.1, gamma=0.01, kernel=rbf;, score=-0.031 total time=   0.0s
[CV 2/5] END ....C=0.1, gamma=0.01, kernel=rbf;

,estimator,SVR()
,param_grid,"{'C': [0.1, 1, ...], 'gamma': [1, 0.1, ...], 'kernel': ['rbf']}"
,scoring,None
,n_jobs,None
,refit,True
,cv,None
,verbose,3
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,kernel,'rbf'


In [52]:
grid.best_score_
grid.best_params_

{'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}

In [53]:
grid_prediction=grid.predict(x_test)

In [54]:
from sklearn.metrics import r2_score,mean_absolute_error
print(mean_absolute_error(y_test,grid_prediction))
print(r2_score(y_test,grid_prediction))

4.701706741577236
0.38694403251274045
